# Lagrangian and Hamiltonian Mechanics

**SIIEA Quantum Engineering Curriculum**
- **Curriculum Days:** Days 141-168
- **License:** CC BY-NC-SA 4.0 | Siiea Innovations, LLC

---

In [ ]:
# Hardware detection — adapts simulations to your machine
import sys, os
sys.path.insert(0, os.path.join(os.path.dirname(os.path.abspath("__file__")), ".."))
try:
    from hardware_config import HARDWARE, get_max_qubits
    print(f"Hardware: {HARDWARE['chip']} | {HARDWARE['memory_gb']} GB | Profile: {HARDWARE['profile']}")
    print(f"Max qubits: {get_max_qubits('safe')} (safe) / {get_max_qubits('max')} (max)")
except ImportError:
    print("hardware_config.py not found — using defaults")
    print("Run setup.sh from the repo root to generate it")

In [ ]:
# Standard scientific stack
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import sympy as sp
from sympy import symbols, Function, cos, sin, diff, simplify, latex, Matrix
from sympy import solve, Rational, pi, sqrt, Eq, pprint
from scipy.integrate import solve_ivp
%matplotlib inline

# Publication-quality plot defaults
plt.rcParams.update({
    'figure.figsize': (8, 6),
    'font.size': 12,
    'axes.titlesize': 14,
    'axes.labelsize': 12,
    'lines.linewidth': 2,
    'figure.dpi': 100,
})
print("Imports loaded. Ready for classical mechanics explorations.")

## From Classical to Quantum: Why Hamiltonian Mechanics Matters

Classical mechanics has three equivalent formulations:
1. **Newtonian:** $\mathbf{F} = m\mathbf{a}$ (forces and accelerations)
2. **Lagrangian:** $\mathcal{L} = T - V$ (energy-based, generalized coordinates)
3. **Hamiltonian:** $H = T + V$ (phase space, canonical coordinates)

The **Hamiltonian formulation** is the direct bridge to quantum mechanics:

| Classical (Hamiltonian) | Quantum Mechanics |
|-------------------------|-------------------|
| $H(q, p)$ | $\hat{H}$ (Hamiltonian operator) |
| $\{f, g\}_{\text{Poisson}}$ | $\frac{1}{i\hbar}[\hat{f}, \hat{g}]$ (commutator) |
| Phase space $(q, p)$ | Hilbert space $|\psi\rangle$ |
| Hamilton's equations | Heisenberg equations of motion |
| $\frac{df}{dt} = \{f, H\}$ | $\frac{d\hat{f}}{dt} = \frac{1}{i\hbar}[\hat{f}, \hat{H}]$ |

This notebook develops all three formulations, culminating in the Poisson bracket --
commutator correspondence that is the classical-quantum bridge.

## 1. Newton's Laws: Projectile with Air Resistance

The simplest formulation: $\mathbf{F} = m\mathbf{a}$.

For a projectile with quadratic air resistance (drag):
$$m\ddot{x} = -b\dot{x}|\mathbf{v}|, \qquad m\ddot{y} = -mg - b\dot{y}|\mathbf{v}|$$

where $|\mathbf{v}| = \sqrt{\dot{x}^2 + \dot{y}^2}$ and $b$ is the drag coefficient.

This ODE system has no closed-form solution, so we solve it numerically.

In [ ]:
# --- Projectile motion with and without air resistance ---

def projectile_ode(t, state, g=9.81, b_over_m=0.0):
    """
    ODE for 2D projectile with quadratic drag.
    state = [x, y, vx, vy]
    b_over_m = drag coefficient / mass
    """
    x, y, vx, vy = state
    v = np.sqrt(vx**2 + vy**2)
    drag_x = -b_over_m * vx * v
    drag_y = -b_over_m * vy * v
    return [vx, vy, drag_x, -g + drag_y]

# Initial conditions: 45-degree launch at 30 m/s
v0 = 30.0
angle = np.radians(45)
y0 = [0, 0, v0 * np.cos(angle), v0 * np.sin(angle)]

# Solve for different drag coefficients
t_span = (0, 8)
t_eval = np.linspace(0, 8, 1000)

# Event to detect ground contact (y = 0 after launch)
def hit_ground(t, state, g=9.81, b_over_m=0.0):
    return state[1]  # y coordinate
hit_ground.terminal = True
hit_ground.direction = -1

drag_values = [0.0, 0.01, 0.03, 0.05]
colors = ['blue', 'green', 'orange', 'red']

fig, axes = plt.subplots(1, 2, figsize=(15, 6))

for b, color in zip(drag_values, colors):
    # Create a closure for the event function with the right drag value
    def event_func(t, state, g=9.81, b_over_m=b):
        return state[1]
    event_func.terminal = True
    event_func.direction = -1

    sol = solve_ivp(
        lambda t, y: projectile_ode(t, y, b_over_m=b),
        t_span, y0, t_eval=t_eval, events=event_func,
        max_step=0.01, dense_output=True
    )

    # Mask out underground portions
    mask = sol.y[1] >= -0.1
    label = f"b/m = {b}" if b > 0 else "No drag"

    axes[0].plot(sol.y[0][mask], sol.y[1][mask], '-', color=color, label=label)

    # Speed vs time
    speed = np.sqrt(sol.y[2]**2 + sol.y[3]**2)
    axes[1].plot(sol.t[mask], speed[mask], '-', color=color, label=label)

# Trajectory plot
axes[0].set_xlabel('Horizontal distance (m)')
axes[0].set_ylabel('Height (m)')
axes[0].set_title('Projectile Trajectories')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim(bottom=-1)

# Speed plot
axes[1].set_xlabel('Time (s)')
axes[1].set_ylabel('Speed (m/s)')
axes[1].set_title('Speed vs Time')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Compare ranges
print("=== Range comparison ===")
for b in drag_values:
    sol = solve_ivp(
        lambda t, y, b=b: projectile_ode(t, y, b_over_m=b),
        t_span, y0, events=hit_ground, max_step=0.01
    )
    x_range = sol.y[0, -1] if len(sol.y[0]) > 0 else 0
    t_flight = sol.t[-1]
    label = f"b/m={b}" if b > 0 else "no drag"
    print(f"  {label:>10s}: range = {x_range:.2f} m, flight time = {t_flight:.2f} s")

## 2. Lagrangian Mechanics

The Lagrangian $\mathcal{L} = T - V$ encodes the dynamics of a system.
The **Euler-Lagrange equation** yields the equations of motion:

$$\frac{d}{dt}\frac{\partial \mathcal{L}}{\partial \dot{q}_i} - \frac{\partial \mathcal{L}}{\partial q_i} = 0$$

**Advantages over Newton:**
- Works in *any* coordinate system (generalized coordinates)
- Constraints are handled naturally
- Symmetries directly imply conservation laws (Noether's theorem)

**Example:** Simple pendulum with generalized coordinate $\theta$:
$$\mathcal{L} = \frac{1}{2}ml^2\dot{\theta}^2 - mgl(1 - \cos\theta)$$

In [ ]:
# --- Lagrangian mechanics: derive equations of motion symbolically ---

# Define symbols
t = symbols('t')
m, l, g_sym = symbols('m l g', positive=True)

# theta is a function of time
theta = Function('theta')(t)
theta_dot = diff(theta, t)
theta_ddot = diff(theta, t, 2)

# Simple pendulum Lagrangian
T = Rational(1, 2) * m * l**2 * theta_dot**2      # Kinetic energy
V = m * g_sym * l * (1 - cos(theta))               # Potential energy
L = T - V

print("=== Simple Pendulum ===")
print(f"T (kinetic energy)  = {T}")
print(f"V (potential energy) = {V}")
print(f"L = T - V = {simplify(L)}")

# Euler-Lagrange equation: d/dt(dL/d(theta_dot)) - dL/d(theta) = 0
dL_dtheta_dot = diff(L, theta_dot)
dL_dtheta = diff(L, theta)
EL = diff(dL_dtheta_dot, t) - dL_dtheta

print(f"\nd/dt(dL/d(theta_dot)) = {simplify(diff(dL_dtheta_dot, t))}")
print(f"dL/d(theta) = {simplify(dL_dtheta)}")

EL_simplified = simplify(EL)
print(f"\nEuler-Lagrange equation:")
print(f"  {EL_simplified} = 0")

# Solve for theta_ddot
eq_of_motion = solve(EL_simplified, theta_ddot)
print(f"\nEquation of motion:")
print(f"  theta_ddot = {eq_of_motion[0]}")
print(f"\nSmall-angle approximation (sin(theta) ~ theta):")
print(f"  theta_ddot = -(g/l) * theta  -->  SHO with omega = sqrt(g/l)")

# ---- Now do the double pendulum ----
print("\n\n=== Double Pendulum (symbolic derivation) ===")
m1, m2, l1, l2 = symbols('m_1 m_2 l_1 l_2', positive=True)
theta1 = Function('theta_1')(t)
theta2 = Function('theta_2')(t)
th1_dot = diff(theta1, t)
th2_dot = diff(theta2, t)

# Positions of masses
x1 = l1 * sin(theta1)
y1 = -l1 * cos(theta1)
x2 = x1 + l2 * sin(theta2)
y2 = y1 - l2 * cos(theta2)

# Velocities (squared)
v1_sq = simplify(diff(x1, t)**2 + diff(y1, t)**2)
v2_sq = simplify(diff(x2, t)**2 + diff(y2, t)**2)

# Lagrangian
T_double = Rational(1, 2) * m1 * v1_sq + Rational(1, 2) * m2 * v2_sq
V_double = m1 * g_sym * y1 + m2 * g_sym * y2
L_double = T_double - V_double

print(f"T = {simplify(T_double)}")
print(f"\n(Full Lagrangian is complex - showing structure)")
print(f"Number of terms in L: ~{len(str(simplify(L_double)).split('+'))} terms")
print("\nThe double pendulum has coupled, nonlinear equations of motion.")
print("We will simulate it numerically next.")

## 3. Double Pendulum: Chaos in Classical Mechanics

The double pendulum is one of the simplest systems exhibiting **deterministic chaos**.
Tiny differences in initial conditions lead to vastly different trajectories.

The equations of motion (derived from the Lagrangian above) are coupled nonlinear
ODEs that must be solved numerically. We use `scipy.integrate.solve_ivp` with the
RK45 method.

In [ ]:
# --- Double Pendulum Simulation ---

def double_pendulum_ode(t, state, m1=1.0, m2=1.0, l1=1.0, l2=1.0, g=9.81):
    """
    Equations of motion for the double pendulum.
    state = [theta1, theta2, omega1, omega2]
    """
    th1, th2, w1, w2 = state
    delta = th1 - th2
    den = m1 + m2 * np.sin(delta)**2

    # Angular accelerations (from Lagrangian mechanics)
    alpha1 = (-m2 * l1 * w1**2 * np.sin(delta) * np.cos(delta)
              - m2 * l2 * w2**2 * np.sin(delta)
              - (m1 + m2) * g * np.sin(th1)
              + m2 * g * np.sin(th2) * np.cos(delta)) / (l1 * den)

    alpha2 = ((m1 + m2) * l1 * w1**2 * np.sin(delta)
              + (m1 + m2) * g * np.sin(th1) * np.cos(delta)
              - (m1 + m2) * l2 * w2**2 * np.sin(delta) * np.cos(delta)
              - (m1 + m2) * g * np.sin(th2)) / (l2 * den)

    return [w1, w2, alpha1, alpha2]

# Simulate two nearby initial conditions (to show chaos)
l1, l2 = 1.0, 1.0
t_span = (0, 20)
t_eval = np.linspace(0, 20, 5000)

# Initial conditions: slightly different starting angles
ic1 = [np.pi/2, np.pi/2, 0, 0]          # theta1=90, theta2=90
ic2 = [np.pi/2 + 0.001, np.pi/2, 0, 0]  # theta1=90.06deg, theta2=90

sol1 = solve_ivp(double_pendulum_ode, t_span, ic1, t_eval=t_eval, max_step=0.005)
sol2 = solve_ivp(double_pendulum_ode, t_span, ic2, t_eval=t_eval, max_step=0.005)

# Convert to Cartesian for plotting
def pendulum_xy(sol, l1=1.0, l2=1.0):
    th1, th2 = sol.y[0], sol.y[1]
    x1 = l1 * np.sin(th1)
    y1 = -l1 * np.cos(th1)
    x2 = x1 + l2 * np.sin(th2)
    y2 = y1 - l2 * np.cos(th2)
    return x1, y1, x2, y2

x1_a, y1_a, x2_a, y2_a = pendulum_xy(sol1)
x1_b, y1_b, x2_b, y2_b = pendulum_xy(sol2)

fig, axes = plt.subplots(1, 3, figsize=(18, 5.5))

# Trace of second mass
axes[0].plot(x2_a, y2_a, 'b-', alpha=0.4, linewidth=0.5, label='IC 1')
axes[0].plot(x2_b, y2_b, 'r-', alpha=0.4, linewidth=0.5, label='IC 2 (+0.001 rad)')
axes[0].set_xlabel('x (m)')
axes[0].set_ylabel('y (m)')
axes[0].set_title('Trace of Second Mass')
axes[0].set_aspect('equal')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Angle difference divergence
angle_diff = np.abs(sol1.y[0] - sol2.y[0])
axes[1].semilogy(sol1.t, angle_diff, 'k-', linewidth=1)
axes[1].set_xlabel('Time (s)')
axes[1].set_ylabel('$|\Delta\theta_1|$ (rad)')
axes[1].set_title('Sensitive Dependence on Initial Conditions')
axes[1].grid(True, alpha=0.3)
axes[1].axhline(0.001, color='red', linestyle='--', alpha=0.5, label='Initial difference')
axes[1].legend()

# theta1 vs time comparison
axes[2].plot(sol1.t, sol1.y[0], 'b-', alpha=0.7, linewidth=1, label='IC 1')
axes[2].plot(sol2.t, sol2.y[0], 'r-', alpha=0.7, linewidth=1, label='IC 2')
axes[2].set_xlabel('Time (s)')
axes[2].set_ylabel('$\theta_1$ (rad)')
axes[2].set_title('$\theta_1$ Divergence Over Time')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.suptitle('Double Pendulum: Deterministic Chaos', fontsize=15, y=1.02)
plt.tight_layout()
plt.show()

print(f"Initial angle difference: {0.001:.4f} rad = {np.degrees(0.001):.4f} deg")
print(f"Final angle difference:   {angle_diff[-1]:.4f} rad = {np.degrees(angle_diff[-1]):.2f} deg")

## 4. Hamiltonian Mechanics and Phase Space

The **Hamiltonian** is obtained from the Lagrangian by a Legendre transform:

$$p_i = \frac{\partial \mathcal{L}}{\partial \dot{q}_i}, \qquad H(q, p) = \sum_i p_i \dot{q}_i - \mathcal{L}$$

**Hamilton's equations** (first-order ODEs):

$$\dot{q}_i = \frac{\partial H}{\partial p_i}, \qquad \dot{p}_i = -\frac{\partial H}{\partial q_i}$$

The system evolves in **phase space** $(q, p)$, and Liouville's theorem guarantees
that phase-space volume is preserved (incompressible flow).

For the simple harmonic oscillator: $H = \frac{p^2}{2m} + \frac{1}{2}kq^2$

Phase-space trajectories are **ellipses** (constant energy curves).

In [ ]:
# --- Hamiltonian formulation: phase space of the harmonic oscillator ---

# Symbolic Hamiltonian for SHO
q, p_sym = symbols('q p')
m_val, k_val = symbols('m k', positive=True)

H_sho = p_sym**2 / (2 * m_val) + Rational(1, 2) * k_val * q**2
print("=== Simple Harmonic Oscillator ===")
print(f"H(q, p) = {H_sho}")

# Hamilton's equations
q_dot = diff(H_sho, p_sym)
p_dot = -diff(H_sho, q)
print(f"dq/dt = dH/dp = {q_dot}")
print(f"dp/dt = -dH/dq = {p_dot}")

# Numerical phase space visualization
m_num, k_num = 1.0, 4.0  # omega = sqrt(k/m) = 2
omega = np.sqrt(k_num / m_num)

def sho_hamiltonian(q, p, m=1.0, k=4.0):
    return p**2 / (2*m) + 0.5 * k * q**2

# Phase portrait: vector field + trajectories
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Left: phase portrait with trajectories
ax = axes[0]
q_range = np.linspace(-3, 3, 20)
p_range = np.linspace(-6, 6, 20)
Q, P = np.meshgrid(q_range, p_range)

# Vector field from Hamilton's equations
dQ = P / m_num       # dq/dt = p/m
dP = -k_num * Q      # dp/dt = -kq

# Normalize arrows for visualization
speed = np.sqrt(dQ**2 + dP**2)
speed[speed == 0] = 1
ax.quiver(Q, P, dQ/speed, dP/speed, speed, cmap='coolwarm', alpha=0.6)

# Plot several trajectories (different energies)
t_traj = np.linspace(0, 2 * np.pi / omega, 500)
for E in [0.5, 2.0, 4.5, 8.0]:
    # For SHO: q(t) = A*cos(wt), p(t) = -m*w*A*sin(wt)
    A = np.sqrt(2 * E / k_num)
    q_traj = A * np.cos(omega * t_traj)
    p_traj = -m_num * omega * A * np.sin(omega * t_traj)
    ax.plot(q_traj, p_traj, '-', linewidth=2, label=f'E = {E}')

ax.set_xlabel('Position $q$')
ax.set_ylabel('Momentum $p$')
ax.set_title('Phase Space: Harmonic Oscillator')
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)
ax.set_aspect('auto')

# Right: energy contour plot
ax = axes[1]
q_fine = np.linspace(-3, 3, 200)
p_fine = np.linspace(-6, 6, 200)
Q_fine, P_fine = np.meshgrid(q_fine, p_fine)
H_vals = sho_hamiltonian(Q_fine, P_fine, m_num, k_num)

contour = ax.contourf(Q_fine, P_fine, H_vals, levels=20, cmap='viridis')
plt.colorbar(contour, ax=ax, label='Energy $H(q, p)$')
ax.contour(Q_fine, P_fine, H_vals, levels=10, colors='white', linewidths=0.5, alpha=0.5)
ax.set_xlabel('Position $q$')
ax.set_ylabel('Momentum $p$')
ax.set_title('Energy Contours in Phase Space')
ax.set_aspect('auto')

plt.tight_layout()
plt.show()

print(f"omega = sqrt(k/m) = {omega:.4f} rad/s")
print(f"Period = 2*pi/omega = {2*np.pi/omega:.4f} s")
print("Phase space orbits are CLOSED (energy is conserved).")
print("Area is preserved (Liouville's theorem).")

## 5. Conservation Laws Verified Numerically

**Noether's Theorem** connects symmetries to conservation laws:

| Symmetry | Conserved Quantity |
|----------|-------------------|
| Time translation | Energy |
| Space translation | Linear momentum |
| Rotational | Angular momentum |

For the simple pendulum, the Hamiltonian (total energy) is conserved:
$$H = \frac{p_\theta^2}{2ml^2} + mgl(1 - \cos\theta)$$

In [ ]:
# --- Verify conservation of energy in simple pendulum simulation ---

def pendulum_ode(t, state, g=9.81, l=1.0):
    """Simple pendulum ODE: state = [theta, omega]"""
    theta, omega = state
    return [omega, -(g/l) * np.sin(theta)]

def pendulum_energy(theta, omega, m=1.0, g=9.81, l=1.0):
    """Total energy: T + V"""
    T = 0.5 * m * l**2 * omega**2
    V = m * g * l * (1 - np.cos(theta))
    return T, V, T + V

# Simulate pendulum with large initial angle (nonlinear regime)
g_val, l_val, m_val = 9.81, 1.0, 1.0
theta0 = np.pi * 0.9  # Nearly vertical (162 degrees)
ic = [theta0, 0.0]

sol = solve_ivp(pendulum_ode, (0, 10), ic, t_eval=np.linspace(0, 10, 2000),
                max_step=0.005, args=(g_val, l_val))

# Compute energies
T, V, E_total = pendulum_energy(sol.y[0], sol.y[1], m_val, g_val, l_val)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Angle vs time
axes[0, 0].plot(sol.t, np.degrees(sol.y[0]), 'b-')
axes[0, 0].set_xlabel('Time (s)')
axes[0, 0].set_ylabel('$\theta$ (degrees)')
axes[0, 0].set_title(f'Pendulum Angle ($\theta_0 = {np.degrees(theta0):.0f}°$)')
axes[0, 0].grid(True, alpha=0.3)

# Phase portrait
axes[0, 1].plot(sol.y[0], sol.y[1], 'r-', linewidth=1)
axes[0, 1].set_xlabel('$\theta$ (rad)')
axes[0, 1].set_ylabel('$\omega$ (rad/s)')
axes[0, 1].set_title('Phase Portrait')
axes[0, 1].grid(True, alpha=0.3)

# Energy components
axes[1, 0].plot(sol.t, T, 'r-', label='Kinetic $T$', alpha=0.7)
axes[1, 0].plot(sol.t, V, 'b-', label='Potential $V$', alpha=0.7)
axes[1, 0].plot(sol.t, E_total, 'k--', linewidth=2, label='Total $E = T + V$')
axes[1, 0].set_xlabel('Time (s)')
axes[1, 0].set_ylabel('Energy (J)')
axes[1, 0].set_title('Energy Conservation')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Energy error (deviation from initial)
E_error = np.abs(E_total - E_total[0]) / E_total[0]
axes[1, 1].semilogy(sol.t, E_error + 1e-16, 'k-', linewidth=1)
axes[1, 1].set_xlabel('Time (s)')
axes[1, 1].set_ylabel('Relative Energy Error')
axes[1, 1].set_title('Numerical Energy Conservation')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Initial energy:     E(0) = {E_total[0]:.6f} J")
print(f"Final energy:       E(T) = {E_total[-1]:.6f} J")
print(f"Max relative error: {np.max(E_error):.2e}")
print(f"Energy is conserved to ~{np.max(E_error):.1e} (limited by numerical integrator)")

## 6. Poisson Brackets: The Classical-Quantum Bridge

The **Poisson bracket** of two phase-space functions $f(q, p)$ and $g(q, p)$ is:

$$\{f, g\} = \sum_i \left( \frac{\partial f}{\partial q_i}\frac{\partial g}{\partial p_i} - \frac{\partial f}{\partial p_i}\frac{\partial g}{\partial q_i} \right)$$

**Fundamental Poisson brackets:**
$$\{q_i, q_j\} = 0, \qquad \{p_i, p_j\} = 0, \qquad \{q_i, p_j\} = \delta_{ij}$$

**The bridge to quantum mechanics (Dirac's prescription):**

$$\{f, g\}_{\text{Poisson}} \longrightarrow \frac{1}{i\hbar}[\hat{f}, \hat{g}]$$

This means $\{q, p\} = 1$ becomes $[\hat{q}, \hat{p}] = i\hbar$, the
**canonical commutation relation** --- the foundation of all quantum mechanics.

In [ ]:
# --- Poisson Brackets with SymPy ---

q1, q2, p1, p2 = symbols('q_1 q_2 p_1 p_2')

def poisson_bracket(f, g, q_vars, p_vars):
    """
    Compute the Poisson bracket {f, g} = sum (df/dqi * dg/dpi - df/dpi * dg/dqi).

    Parameters
    ----------
    f, g : sympy expressions
    q_vars : list of position symbols
    p_vars : list of momentum symbols

    Returns
    -------
    sympy expression for {f, g}
    """
    result = 0
    for qi, pi in zip(q_vars, p_vars):
        result += diff(f, qi) * diff(g, pi) - diff(f, pi) * diff(g, qi)
    return simplify(result)


# One degree of freedom
q_vars_1d = [q1]
p_vars_1d = [p1]

print("=== Fundamental Poisson Brackets (1D) ===")
print(f"  {{q, p}} = {poisson_bracket(q1, p1, q_vars_1d, p_vars_1d)}")
print(f"  {{q, q}} = {poisson_bracket(q1, q1, q_vars_1d, p_vars_1d)}")
print(f"  {{p, p}} = {poisson_bracket(p1, p1, q_vars_1d, p_vars_1d)}")

# SHO Hamiltonian
m_sym, k_sym = symbols('m k', positive=True)
H = p1**2 / (2 * m_sym) + Rational(1, 2) * k_sym * q1**2
print(f"\n=== SHO Hamiltonian: H = {H} ===")
print(f"  {{q, H}} = {poisson_bracket(q1, H, q_vars_1d, p_vars_1d)}  (= dq/dt = p/m)")
print(f"  {{p, H}} = {poisson_bracket(p1, H, q_vars_1d, p_vars_1d)}  (= dp/dt = -kq)")

# Angular momentum example (2D)
q_vars_2d = [q1, q2]
p_vars_2d = [p1, p2]
L_z = q1 * p2 - q2 * p1  # Angular momentum

print(f"\n=== Angular Momentum L_z = {L_z} ===")
print(f"  {{L_z, q1}} = {poisson_bracket(L_z, q1, q_vars_2d, p_vars_2d)}")
print(f"  {{L_z, q2}} = {poisson_bracket(L_z, q2, q_vars_2d, p_vars_2d)}")
print(f"  {{L_z, p1}} = {poisson_bracket(L_z, p1, q_vars_2d, p_vars_2d)}")
print(f"  {{L_z, p2}} = {poisson_bracket(L_z, p2, q_vars_2d, p_vars_2d)}")

# 2D isotropic oscillator
H_2d = (p1**2 + p2**2) / (2 * m_sym) + Rational(1, 2) * k_sym * (q1**2 + q2**2)
print(f"\n=== 2D Isotropic Oscillator ===")
print(f"  H = {H_2d}")
print(f"  {{L_z, H}} = {poisson_bracket(L_z, H_2d, q_vars_2d, p_vars_2d)}  (L_z conserved!)")

print("\n=== Classical-Quantum Correspondence ===")
print("  {q, p} = 1          -->  [q_hat, p_hat] = i*hbar")
print("  {f, H} = df/dt      -->  [f_hat, H_hat] / (i*hbar) = df_hat/dt")
print("  {L_z, H} = 0        -->  [L_z_hat, H_hat] = 0 (simultaneous eigenstates)")

## Summary

| Classical Concept | Formula | Quantum Analog |
|-------------------|---------|----------------|
| Lagrangian | $\mathcal{L} = T - V$ | Path integral $\int e^{iS/\hbar} \mathcal{D}[q]$ |
| Euler-Lagrange | $\frac{d}{dt}\frac{\partial L}{\partial \dot{q}} = \frac{\partial L}{\partial q}$ | Heisenberg equation |
| Hamiltonian | $H = T + V$ | $\hat{H}$ operator |
| Hamilton's eqs | $\dot{q} = \partial H/\partial p$ | $i\hbar \partial_t|\psi\rangle = \hat{H}|\psi\rangle$ |
| Poisson bracket | $\{q, p\} = 1$ | $[\hat{q}, \hat{p}] = i\hbar$ |
| Phase space | $(q, p)$ | Hilbert space $|\psi\rangle$ |

**Key Takeaways:**
1. Lagrangian mechanics replaces forces with energy --- more elegant and general
2. The double pendulum demonstrates that classical mechanics can be chaotic
3. Hamiltonian mechanics reveals phase-space structure preserved by evolution
4. Conservation laws follow from symmetries (Noether's theorem)
5. Poisson brackets map directly to quantum commutators (Dirac's prescription)
6. The Hamiltonian formulation is **the** starting point for quantization

---
*This completes the classical foundations. Next: Year 0 continues with complex analysis,
electromagnetism, and functional analysis --- all building toward quantum mechanics in Year 1.*